In [ ]:
from loop_loss_rl import training_loop_rl
from NER_cadec import EntityMatrixPredictor
import torch

model = EntityMatrixPredictor(bert_model_name='bert-base-cased')
model.load_state_dict(torch.load("./models/cadec/model_17.pth"))
model = training_loop_rl("./datasets/cadec", model=model, verbose=True, pos_weight=17, epochs=1, rl_weight=10)

Reward: 1.5438597202301025
Reward: 1.6666667461395264
Reward: 1.2017543315887451
Reward: 1.675438642501831
Reward: 1.5789473056793213
Reward: 1.8947367668151855
Reward: 1.824561357498169
Reward: 0.8508771657943726
Reward: 1.4210525751113892
Reward: 1.1403508186340332
Reward: 1.5877193212509155
Reward: 1.8157894611358643
Reward: 1.6140351295471191
Reward: 1.9912281036376953
Reward: 1.6666667461395264
Reward: 1.4649121761322021
Reward: 0.9736841917037964
Reward: 1.4824562072753906
Reward: 1.5701754093170166
Reward: 1.9210526943206787
Reward: 1.736842155456543
Reward: 1.8684210777282715
Reward: 1.2456140518188477
Reward: 1.175438642501831
Reward: 1.763157844543457
Reward: 0.9385964870452881
Reward: 1.4649122953414917
Reward: 1.7280702590942383
Reward: 1.9649121761322021
Reward: 0.8947368860244751
Reward: 1.6666667461395264
Reward: 1.4210526943206787
Batch loss 0.5970603823661804
Reward: 1.6810345649719238
Reward: 0.9224137663841248
Reward: 1.6379311084747314
Reward: 1.965517282485962
Rewa

In [5]:
import torch
torch.save(model.state_dict(), "./models/cadec/model_rlp_10.pth")

In [6]:
from NER_cadec import evaluation_loop
model = EntityMatrixPredictor(bert_model_name='bert-base-cased')
model.load_state_dict(torch.load("./models/cadec/model_17.pth"))
print(evaluation_loop("./datasets/cadec", model))
model = EntityMatrixPredictor(bert_model_name='bert-base-cased')
model.load_state_dict(torch.load("./models/cadec/model_rlp_10.pth"))
evaluation_loop("./datasets/cadec", model)

(0.20902511707109409, 0.6680272108843538, 0.31841763942931256)


(0.23741529525653438, 0.6673469387755102, 0.3502320599785791)

In [20]:
from datasets import load_dataset

dataset = load_dataset("ade_corpus_v2")  # Or a specific community version if available
dataset["train"][1:100]

ValueError: Config name is missing.
Please pick one among the available configs: ['Ade_corpus_v2_classification', 'Ade_corpus_v2_drug_ade_relation', 'Ade_corpus_v2_drug_dosage_relation']
Example of usage:
	`load_dataset('ade_corpus_v2', 'Ade_corpus_v2_classification')`

In [275]:
def find_opening_paths(matrix, threshold=0.5):
    """
    Recursively find all maximal opening paths (as sequences of node indices) in the upper triangle of the matrix.

    Args:
        matrix (torch.Tensor): A square matrix of shape (N, N)
        threshold (float): Minimum value to consider an arc active

    Returns:
        List[List[int]]: Only maximal (non-nested) paths represented as lists of node indices.
    """
    n = matrix.size(0)
    all_paths = []

    def recurse(path):
        i,j = path[-1]
        branches = []
        for k in range(i + 1, n):
            if matrix[j, k] > threshold: 
                for m in range(k,n):
                    if matrix[j, m] > threshold: 
                        branches.append((j, m))
                        matrix[j, m] = 0
                break
        if branches == []: 
            all_paths.append(path)
            return
        for j,m in branches:
            recurse(path + [(j,m)])

    for i in range(n):
        for j in range(i + 1, n):
            if matrix[i, j] > threshold:
                recurse([(i, j)])

    return all_paths


In [ ]:
def ordered_partitions(lst):
    if not lst:
        yield []
        return
    for i in range(1, len(lst) + 1):
        first = lst[:i]
        for rest in ordered_partitions(lst[i:]):
            yield [first] + rest

In [240]:
def opening_paths_partitions(path):
    """
    Given a forward path as list of arcs (i, j), generate all valid backward-closing partitions.
    Each partition is a list of arcs like (to, from) that would close the entity.

    Args:
        path: List of tuples representing the forward arcs (e.g., [(0,1), (1,2), (2,3)])

    Returns:
        List[List[Tuple[int, int]]]: All possible backward arcs to close the path
    """
    repairs = []
    for p in ordered_partitions(path):
        rep = []
        for e in p:
            rep.append((e[-1][1], e[0][0]))
        repairs.append(rep)

    return repairs

In [283]:
def repair_matrix_lower(logits, threshold=0.5):
    upper = torch.triu(logits, diagonal=1)
    paths = find_opening_paths(logits, threshold)
    print(paths)
    for path in paths:
        partitions = opening_paths_partitions(path)
        partitions_weight = []
        for p in partitions:
            partitions_weight.append(sum(list(map(lambda cell: 1-logits[cell[0],cell[1]], p))))

        for repaired_cell in partitions[partitions_weight.index(min(partitions_weight))]:
            logits[repaired_cell[0], repaired_cell[1]] = 1
    return torch.tril(logits, diagonal=-1) + upper

In [290]:
logits = torch.tensor([
    [0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.9, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.8],
    [0.0, 0.0, 0.0, 0.0, 0.0],
])

repair_matrix_lower(logits, threshold=0.5)

[[(1, 3), (3, 4)]]


tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.9000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.8000],
        [0.0000, 1.0000, 0.0000, 0.0000, 0.0000]])

In [285]:
def find_closing_paths(logits, treshold=0.5):
    """
    Recursively find all maximal closing paths (as sequences of node indices) in the lower triangle of the matrix.

    Args:
        matrix (torch.Tensor): A square matrix of shape (N, N)
        threshold (float): Minimum value to consider an arc active

    Returns:
        List[List[int]]: Only maximal (non-nested) paths represented as lists of node indices.
    """
    n = logits.size(0)
    all_paths = []
    for i in range(n):
        for j in range(i + 1, n):
            if logits[j, i] > treshold:
                all_paths.append((j,i))

    return all_paths

In [344]:
def closing_path_partitions(path):
    i,j = path
    partitions = []
    for p in ordered_partitions(list(range(i - j))):
        k,l = j, j
        part = []
        for e in p:
            k = k + len(e)
            part.append((l,k))
            l = l + len(e)
        partitions.append(part)
    return partitions

In [345]:
def repair_matrix_upper(logits, threshold=0.5):
    lower = torch.tril(logits, diagonal=-1)
    paths = find_closing_paths(logits, threshold)
    for path in paths:
        partitions = closing_path_partitions(path)
        partitions_weight = []
        for p in partitions:
            partitions_weight.append(sum(list(map(lambda cell: 1-logits[cell[0],cell[1]], p))))
        for repaired_cell in partitions[partitions_weight.index(min(partitions_weight))]:
            logits[repaired_cell[0], repaired_cell[1]] = 1
    return lower + torch.triu(logits, diagonal=1)

In [347]:
logits = torch.tensor([
    [0.0, 0.8, 0.3, 0.9],
    [0.0, 0.0, 0.9, 0.0],
    [0.0, 0.0, 0.0, 0.8],
    [0.9, 0.0, 0.0, 0.0],
])

repair_matrix_upper(logits, threshold=0.5)

tensor([[0.0000, 0.8000, 0.3000, 1.0000],
        [0.0000, 0.0000, 0.9000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.8000],
        [0.9000, 0.0000, 0.0000, 0.0000]])